Author: Eugene Su

Email: su.eugene@gmail.com

https://sites.google.com/view/smartrobot/lab

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
from google.colab import drive

In [ ]:
# 下載訓練資料和驗證資料
(train_data, val_data), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
# 對圖片正規化
# Min-Max Normalization
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

In [ ]:
# 設定驗證集
val_data = val_data.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_data = val_data.batch(1)  # batch size為1，效能最差，但易於編寫程式
val_data = val_data.cache()
val_data = val_data.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# 掛載Google Drive
drive.mount('/content/drive', force_remount=True)
LOG_ROOT = '/content/drive/My Drive/CIFAR10_CNN'

In [ ]:
model_dir = '20210510-11:22:03-b(64)-lr(0.001)-wd(0.0001)'
checkpoint_name = 'cp-0123-0.9032.ckpt'
full_file_path = os.path.join(LOG_ROOT, model_dir, checkpoint_name)
loaded_model = tf.keras.models.load_model(full_file_path)

In [ ]:
num_val_images = len(val_data)
take_count = 0

# 同學可以將batch size設定為大於1的數值，使用矩陣的思維改寫下方程式碼
for image, label in val_data.take(num_val_images):
    prob = loaded_model.predict(image)
    predicted_class_id = np.argmax(prob)
    ground_truth = label.numpy()
    take_count += 1
    print('{} : predicted_class_id = {}'.format(take_count, predicted_class_id))
    print('{} : ground_truth = {}'.format(take_count, ground_truth))
